In [2]:
import json
import os

In [2]:
data = []
with open('prepared-gov.my-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 100:
            continue
        data.append(l['text'])

In [3]:
len(data)

64500

In [4]:
# !rm -rf rephrase-gov.my-llama3
!mkdir rephrase-gov.my-llama3

mkdir: cannot create directory ‘rephrase-gov.my-llama3’: File exists


In [5]:
results = []
for i in range(len(data)):
    
    s = """
{{replace_me}}

structure text above to become a complete malay wikipedia markdown page, DO NOT BE LAZY, WRITE ALL DOWN EVERYTHING, NO REDACTED AND SO ON, start it with [START] and end with [END]
    """.strip()
    s = s.replace('{{replace_me}}', data[i])
    results.append(s)
        
len(results)

64500

In [6]:
from openai import OpenAI
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

def answer(q, i):
    filename = f'rephrase-gov.my-llama3/{i}.json'
    if os.path.exists(filename):
        return
    
    results = []
    openai = OpenAI(
        base_url='https://llama-3.us.mesolitica.com/v1',
        api_key='empty',
    )
    for _ in range(1):
        
        while True:
            try:
                
                json_data = {
                    'messages': [
                        {
                            'role': 'user',
                            'content': q,
                        },
                    ],
                    'model': 'model',
                    'stop': [
                        '<|eot_id|>',
                    ],
                    'temperature': 0.9,
                    'max_tokens': 8192,
                }
                response = requests.post(
                    'https://llama-3.us.mesolitica.com/v1/chat/completions', 
                    headers=headers, json=json_data, timeout = 60 * 10)
                
                r = response.json()['choices'][0]['message']['content']
                results.append(r)
                break
            except Exception as e:
                # print(e)
                pass
                
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)

In [7]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [8]:
urls = [(q, no) for no, q in enumerate(results)]

In [9]:
answer(*urls[20])

In [10]:
urls[20]

(". Datin Siti Norizah binti Bachik Timbalan Menteri di Jabatan Perdana Menteri (Parlimen dan Undang-Undang) 79. YB Datuk Wira Hajah Mas Ermieyati binti Samsudin YBhg. Encik Mohd Helmy bin Abd. Talib Timbalan Menteri Perdagangan Dalam Negeri dan Hal Ehwal Pengguna 80. YB Dato' Haji Rosol bin Wahid YBhg. To’ Puan Halipah binti Ahmad Timbalan Menteri Dalam Negeri II 81. YB Datuk Jonathan bin Yasin YBhg. Datin Helena binti Masry Timbalan Menteri Pelancongan, Seni dan Budaya 82. YB Datuk Seri Dr. Santhara Kumar a/l Ramanaidu YBhg. Datin Seri Carline a/p A Johnson D’Cruz Timbalan Menteri Alam Sekitar dan Air 83. YB Dato’ Dr. Mansor bin Haji Othman YBhg. Datin Aishah binti Zainul 84. Timbalan Menteri Tenaga, dan Sumber Asli YB Datuk Ali anak Biju Timbalan Menteri Perusahaan Perladangan dan Komoditi I 85. YB Datuk Willie anak Mongin YBhg. Datin Yee Chooi Ling Kemas kini sehingga 12 Oktober 2022 8 Timbalan Menteri Perpaduan Negara 86. YB Senator Tuan Wan Ahmad Fayhsal bin Wan Ahmad Kamal YBhg.

In [12]:
from threading import Thread
from queue import Queue
from tqdm import tqdm

queue = Queue()
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [ ]:
max_worker = 50
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

  2%|█▏                                                                      | 1046/64500 [32:44<347:44:17, 19.73s/it]

In [6]:
data = []
with open('prepared-gov.my-llama3.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if len(l['text']) < 100:
            continue
        data.append(l)

In [8]:
from tqdm import tqdm

with open('gov.my-rephrase.jsonl', 'w') as fopen_l:
    for i in tqdm(range(len(data))):
        filename = f'rephrase-gov.my-llama3/{i}.json'
        if not os.path.exists(filename):
            continue

        with open(filename) as fopen:
            d = json.load(fopen)

        try:
            rephrase = d[0].split('[START]')[1].split('[END]')[0].strip()
            data[i]['rephrase'] = rephrase
            
            fopen_l.write(f'{json.dumps(data[i])}\n')
        except Exception as e:
            pass

100%|███████████████████████████████████████████████████████████████████████| 64500/64500 [00:00<00:00, 130524.48it/s]


In [9]:
!wc -l gov.my-rephrase.jsonl

996 gov.my-rephrase.jsonl


In [11]:
!tail -n 1 gov.my-rephrase.jsonl

{"text": ") lulus semua penilaian tahap kecekapan yang ditetapkan; dan (c) diperaku oleh ketua perkhidmatan. Nota: Skim perkhidmatan ini membatalkan skim perkhidmatan terdahulu yang berkuat kuasa pada 1.10.1995 [Surat Pemakluman Skim Perkhidmatan, JPA(S)324/14/24-1/Klt. 10 (1)]. 673/776 SKIM PERKHIDMATAN JURUPULIH PERUBATAN KLASIFIKASI : PERKHIDMATAN PERUBATAN DAN KESIHATAN KUMPULAN PERKHIDMATAN : SOKONGAN TARIKH KUAT KUASA : 1 NOVEMBER 2002 Gred Jadual Gaji Matriks Gred U29 : P1T1 RM928.55 P2T1 RM985.42 P3T1 RM1044.71 - - - P1T27 RM2445.89 P2T27 RM2623.76 P3T27 RM2781.06 Gred U32 : P1T1 RM1567.43 P2T1 RM1683.59 - - P1T20 RM2925.05 P2T20 RM3227.55 Gred U36 : P1T1 RM2445.89 P2T1 RM2616.50 - - P1T11 RM3617.17 P2T11 RM3999.53 Gred U38 : P1T1 RM3023.06 P2T1 RM3304.99 - - P1T8 RM3954.76 P2T8 RM4312.92 Gred U40 : P1T1 RM3555.46 P2T1 RM3893.05 - - P1T7 RM4513.78 P2T7 RM4923.97 SYARAT LANTIKAN 1. Calon bagi lantikan hendaklah memiliki kelayakan seperti berikut:- (a) warganegara Malaysia; (b) b

In [12]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='gov.my-rephrase.jsonl',
    path_in_repo='gov.my-rephrase.jsonl',
    repo_id='mesolitica/noisy-rephrase',
    repo_type='dataset',
)

gov.my-rephrase.jsonl:   0%|          | 0.00/13.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/noisy-rephrase/commit/82a8f6307867eaa9ccd35c29b227c91ceac91bac', commit_message='Upload gov.my-rephrase.jsonl with huggingface_hub', commit_description='', oid='82a8f6307867eaa9ccd35c29b227c91ceac91bac', pr_url=None, pr_revision=None, pr_num=None)